In [ ]:
#labelme -> "shapes" but we use "array" to get values which also exist in some other format.
#createMl -> "coordinates"
#yolo -> normalized numeric values

In [ ]:
#If there is no bounding box, we should remove the annotation and image. This has to be done 
#since we write good xml file of another class onto the bad xml file which will 
#cause low accuracy during training.

In [ ]:
#For XML files, text will only display the value between tags if there is one. If there are children,
#it won't display contents in between. Just the string value in between tags. 

In [ ]:
#We can combine all three repair files to save time. #Wanted it organized and not jumbled up. 
#But it should be done.

In [1]:
import cv2
from glob import glob
import io
import json
import os
import re
from xml.etree import ElementTree
import xml.etree.cElementTree as ET
import xml.dom.minidom as DM
from xml.parsers.expat import ExpatError

In [251]:
def get_attributes(class_dir,file,image_name):
    features=dict()
    features['bndbox']=[]
    try:
        image = cv2.imread(class_dir+image_name)
        HEIGHT, WIDTH = image.shape[:2]
        tree=ET.parse(class_dir+file)
        root = tree.getroot()
        bndbox = list()
        if(len(root.findall('.//coordinates'))>=1):
            for box in root.findall('.//coordinates'):
                xcenter=((float)(box.find('x').text))
                ycenter=((float)(box.find('y').text))
                width=((float)(box.find('width').text))
                height=((float)(box.find('height').text))
                #Normalized coordinates
                if(xcenter<=1 and ycenter <=1 and width <=1 and height<=1):
                    xmax=((2*xcenter*WIDTH)+(width*WIDTH))/2
                    xmin=xmax-(WIDTH*width)
                    ymax=((2*ycenter*HEIGHT)+(height*HEIGHT))/2
                    ymin=ymax-(height*HEIGHT)
                #If coordinates are not normalized
                else:
                    xmax=(((2*xcenter)+width)/2)
                    ymax=(((2*ycenter)+height)/2)
                    xmin=(2*xcenter-xmax)
                    ymin=(2*ycenter-ymax)
                bndbox.append(str((int)(xmin)))
                bndbox.append(str((int)(ymin)))
                bndbox.append(str((int)(xmax)))
                bndbox.append(str((int)(ymax)))
            features['bndbox']=bndbox
            features['height']=[str(HEIGHT)]
            features['width']=[str(WIDTH)]
    except ET.ParseError:
        try:
            with open(class_dir+file) as f:
                xml=f.read()
        except UnicodeDecodeError:
            with io.open(class_dir+file, mode="r", encoding="utf-8") as f:
                xml=f.read()
        if(len(re.findall(r"<array>(.*?)</array>",xml))<4):
            text=re.sub(r"[^a-zA-Z0-9.]+", ' ', xml)
            if(len(text)>=4):
                features['bndbox']=re.findall(r"points(.*?)group id",text)[0].split()
                features['height']=re.findall(r"imageHeight(.*)imageWidth",text)[0].split()
                features['width']=re.findall(r"imageWidth(.*)",text)[0].split()
        elif(len(re.findall(r"<array>(.*?)</array>",xml))>=4):
            features["height"]=re.findall(r"<imageHeight>(.*?)</imageHeight>",xml)
            features["width"]=re.findall(r"<imageWidth>(.*?)</imageWidth>",xml)
            features["bndbox"]=[str(x) for x in [int(x) for x in [float(x) for x  in re.findall(r"<array>(.*?)</array>",xml)]]]
    return features

In [70]:
def get_json_attributes(class_dir,file,image_name):
    features=dict()
    bndbox = list()
    raw_bndbox=list()
    image = cv2.imread(class_dir+image_name)
    HEIGHT, WIDTH = image.shape[:2]
    with open(class_dir+file, 'r') as j:
        json_file = json.load(j)
    try:
        coordinates,=re.findall(r"coordinates(.*?)}",str(json_file))
        for k in coordinates.split("\n"):
            raw_bndbox.append(re.sub(r"[^0-9.]+", ' ', k))
        raw_bndbox=raw_bndbox[0].split()
    except ValueError:
        features['bndbox']=[]
        return features
    for bound_iter in range(0,len(raw_bndbox)-1,4):
        xcenter=(float)(raw_bndbox[0+bound_iter])
        ycenter=(float)(raw_bndbox[1+bound_iter])
        width=(float)(raw_bndbox[2+bound_iter])
        height=(float)(raw_bndbox[3+bound_iter])
        #Normalized coordinates
        if(xcenter<=1 and ycenter <=1 and width <=1 and height<=1):
            xmax=((2*xcenter*WIDTH)+(width*WIDTH))/2
            xmin=xmax-(WIDTH*width)
            ymax=((2*ycenter*HEIGHT)+(height*HEIGHT))/2
            ymin=ymax-(height*HEIGHT)
        #If coordinates are not normalized
        else:
            xmax=(((2*xcenter)+width)/2)
            ymax=(((2*ycenter)+height)/2)
            xmin=(2*xcenter-xmax)
            ymin=(2*ycenter-ymax)
        bndbox.append(str((int)(xmin)))
        bndbox.append(str((int)(ymin)))
        bndbox.append(str((int)(xmax)))
        bndbox.append(str((int)(ymax)))
    features['bndbox']=bndbox
    features['height']=[str(HEIGHT)]
    features['width']=[str(WIDTH)]
    return features

In [32]:
def write_xml(class_dir,file,image_name,features):
    os.chdir("C:/Users/arviv/Documents/Programming/Important Files 2/General/Internship - Analysed/Data Preprocessing/")
    good_tree = ElementTree.parse("good_xml.xml")
    good_root = good_tree.getroot()
    for box in range((int)(len(features['bndbox'])/4)-1):
        good_root.append(good_root.find(".//object"))
    good_tree.write(class_dir+file)
    tree = ElementTree.parse(class_dir+file)
    root = tree.getroot()
    count=0
    for box in root.findall('.//object'):
        box.find('.//bndbox/xmin').text = features['bndbox'][0+count*4]
        box.find('.//bndbox/ymin').text = features['bndbox'][1+count*4]
        box.find('.//bndbox/xmax').text = features['bndbox'][2+count*4]
        box.find('.//bndbox/ymax').text = features['bndbox'][3+count*4]
        box.find('.//name').text = " ".join((os.path.splitext(file)[0]).split()[:-1])
        count+=1
    root.find('.//filename').text = image_name
    root.find('.//height').text = features['height'][0]
    root.find('.//width').text = features['width'][0]
    root.find('.//path').text=class_dir
    root.find('.//folder').text=class_dir #name.replace(class_dir[:-1]+"\\","")
    #root.find('.//folder').text=" ".join((os.path.splitext(file)[0]).split()[:-1])
    tree.write(class_dir+file)
    #print(class_dir+file)

In [2]:
#Some unreadable xml files have multiple objects. This is written considering that fact.
def xml_repair(source_dir):
    BAD_LIST=[]
    classes=next(os.walk(source_dir))[1]
    for i in classes:
        class_dir=source_dir+i+'/'
        file_list=glob(class_dir+"*.xml")
        file_list=[name.replace(class_dir[:-1]+"\\","") for name in file_list]            
        #bad_list=inconvertible_xml(class_dir)
        #file_list=[x for x in file_list if x not in bad_list]
        #Image files are found using glob where if they have extension belonging to image_extension_list
        #they are added to image_list. The problem being image_extension includes .jpg and .JPG which act 
        #as the same extension and can cause duplicates. Instead of removing that extension 
        #from the extension list, I want to remove the duplicates from the image_list. 
        #This is because, image_extension_list provided us with the different extensions after looping 
        #through the entire dataset. I want to preserve the information that this extension exists 
        #in case of any future issues.
        image_list=[]
        for ext in image_extension_list:
            image_list.extend(glob(class_dir+'*'+ext))
        image_list=[name.replace(class_dir[:-1]+"\\","") for name in image_list]
        #Image list so that we can get image name
        #Remove duplicates from list
        image_list=[*set(image_list)]
        for file in file_list:
            #print(i,file)
            #image_name will be a single element list
            image_name=[x for x in image_list if os.path.splitext(file)[0]==os.path.splitext(x)[0]]
            if(len(image_name)<1):
                continue
            #We use unpacking to extract the single element from list. 
            #Notice trailing comma below
            image_name, = image_name
            #print("In "+i+" , "+image_name+" , "+file)
            try: 
                tree=ET.parse(class_dir+file)
                root = tree.getroot()
                if(len(root.findall('.//object'))>=1):
                    bndbox=list()
                    for box in root.findall('.//object'):
                        #box.find('.//name').text=" ".join(file_name.split()[:-1])
                        #object name as class_name
                        box.find('.//name').text=i
                        bndbox.append(box.find('.//xmin').text)
                        bndbox.append(box.find('.//ymin').text)
                        bndbox.append(box.find('.//xmax').text)
                        bndbox.append(box.find('.//ymax').text)
                    root.find('.//filename').text = image_name
                    root.find('.//path').text=class_dir
                    root.find('.//folder').text=i
                    tree.write(class_dir+file)
                    print(i, file," Readable xml file with bounding box 1 : ",bndbox)
                    if(len(bndbox)<4):
                        print("Problem ",i,file,bndbox)
                        BAD_LIST.append(class_dir+file)
                        continue
                elif(len(root.findall('.//coordinates'))>=1):
                    #Conversion of [xcenter,ycenter,width,height] into [xmin,ymin,xmax,ymax]
                    features=get_attributes(class_dir,file,image_name)
                    #print(features)
                    write_xml(class_dir,file,image_name,features)
                    tree=ET.parse(class_dir+file)
                    root = tree.getroot()
                    for box in root.findall('.//object'):
                        #box.find('.//name').text=" ".join(file_name.split()[:-1])
                        #ERROR??
                        box.find('.//name').text=i
                    root.find('.//filename').text = image_name
                    root.find('.//path').text=class_dir
                    root.find('.//folder').text=i
                    tree.write(class_dir+file)
                    print(i, file," Readable xml file with bounding box 2 : ",features['bndbox'])
                    if(len(features['bndbox'])<4):
                        print("Problem ",i,file,features['bndbox'])
                        BAD_LIST.append(class_dir+file)
                        continue
                else:
                    print("Readable xml file without coordinates or objects",i,file)
                    BAD_LIST.append(class_dir+file)
                    continue
            except ET.ParseError:
                features=get_attributes(class_dir,file,image_name)
                #print(features)
                if(len(features['bndbox'])<4):
                    print("No bounding boxes found in ",file," of ",i)
                    BAD_LIST.append(class_dir+file)
                    continue
                write_xml(class_dir,file,image_name,features)
                tree=ET.parse(class_dir+file)
                root = tree.getroot()
                for box in root.findall('.//object'):
                    box.find('.//name').text=i
                root.find('.//filename').text = image_name
                root.find('.//path').text=class_dir
                root.find('.//folder').text=i
                tree.write(class_dir+file)
                print(i, file," Unreadable xml file with bounding box : ",features['bndbox'])
    return BAD_LIST

In [30]:
def json_repair(source_dir):
    BAD_LIST=[]
    JSON_LIST=[]
    classes=next(os.walk(source_dir))[1]
    for i in classes:
        class_dir=source_dir+i+'/'
        file_list=glob(class_dir+"*.json")
        file_list=[name.replace(class_dir[:-1]+"\\","") for name in file_list]            
        image_list=[]
        for ext in image_extension_list:
            image_list.extend(glob(class_dir+'*'+ext))
        image_list=[name.replace(class_dir[:-1]+"\\","") for name in image_list]
        image_list=[*set(image_list)]
        for file in file_list:
            image_name=[x for x in image_list if os.path.splitext(file)[0]==os.path.splitext(x)[0]]
            if(len(image_name)<1):
                continue
            image_name, = image_name
            features=get_json_attributes(class_dir,file,image_name)
            if(len(features['bndbox'])<4):
                print("Problem ",i,file,features['bndbox'])
                BAD_LIST.append(class_dir+file)
                continue
            write_xml(class_dir,os.path.splitext(file)[0]+".xml",image_name,features)
            tree=ET.parse(class_dir+os.path.splitext(file)[0]+".xml")
            root = tree.getroot()
            for box in root.findall('.//object'):
                box.find('.//name').text=i
            root.find('.//filename').text = image_name
            root.find('.//path').text=class_dir
            root.find('.//folder').text=i
            tree.write(class_dir+os.path.splitext(file)[0]+".xml")
            JSON_LIST.append(class_dir+file)
            print(i, os.path.splitext(file)[0]+".xml"," Readable xml file with bounding box 2 : ",features['bndbox'])
    return BAD_LIST,JSON_LIST

In [69]:
#Only for yolo format
def txt_repair(source_dir):
    BAD_LIST=[]
    TXT_LIST=[]
    classes=next(os.walk(source_dir))[1]
    for i in classes:
        class_dir=source_dir+i+'/'
        file_list=glob(class_dir+"*.txt")
        file_list=[name.replace(class_dir[:-1]+"\\","") for name in file_list]
        image_list=[]
        for ext in image_extension_list:
            image_list.extend(glob(class_dir+'*'+ext))
        image_list=[name.replace(class_dir[:-1]+"\\","") for name in image_list]
        image_list=[*set(image_list)]
        for file in file_list:
            print(file)
            image_name=[x for x in image_list if os.path.splitext(file)[0]==os.path.splitext(x)[0]]
            #If there is no image for the file, don't bother doing the writing. That's the logic. Bad one
            if(len(image_name)<1):
                continue
            image_name, = image_name
            image = cv2.imread(class_dir+image_name)
            HEIGHT, WIDTH = image.shape[:2]
            features=dict()
            features['bndbox']=list()
            BB=list()
            bndbox=list()
            with open(class_dir+file, 'r') as t:
                txt_file = t.read()
            for bb in txt_file.split("\n"):
                if(len(bb)==0):
                    break
                BB.extend(bb.split()[1:5])
                #print(BB)
            if(len(BB)<4):
                print("Problem ",i,file,BB)
                BAD_LIST.append(class_dir+file)
                continue
            for bound_iter in range(0,len(BB)-1,4):
                xcenter=(float)(BB[0+bound_iter])
                ycenter=(float)(BB[1+bound_iter])
                width=(float)(BB[2+bound_iter])
                height=(float)(BB[3+bound_iter])
                #Normalized coordinates
                if(xcenter<=1 and ycenter <=1 and width <=1 and height<=1):
                    xmax=((2*xcenter*WIDTH)+(width*WIDTH))/2
                    xmin=xmax-(WIDTH*width)
                    ymax=((2*ycenter*HEIGHT)+(height*HEIGHT))/2
                    ymin=ymax-(height*HEIGHT)
                    bndbox.append(str((int)(xmin)))
                    bndbox.append(str((int)(ymin)))
                    bndbox.append(str((int)(xmax)))
                    bndbox.append(str((int)(ymax)))
                else:
                    continue
            features['bndbox']=bndbox
            features['height']=[str(HEIGHT)]
            features['width']=[str(WIDTH)]
            print(features)
            write_xml(class_dir,os.path.splitext(file)[0]+".xml",image_name,features)
            tree=ET.parse(class_dir+os.path.splitext(file)[0]+".xml")
            root = tree.getroot()
            for box in root.findall('.//object'):
                box.find('.//name').text=i
            root.find('.//filename').text = image_name
            root.find('.//path').text=class_dir
            root.find('.//folder').text=i
            tree.write(class_dir+os.path.splitext(file)[0]+".xml")
            TXT_LIST.append(class_dir+file)
            print(i, os.path.splitext(file)[0]+".xml"," Readable xml file with bounding box 2 : ",features['bndbox'])
    return BAD_LIST,TXT_LIST

In [4]:
ROOTDIR= 'D:/Downloads/Analysed/Dataset 3/'
CLASSES=next(os.walk(ROOTDIR))[1]
disparity_dir='D:/Downloads/Analysed/Disparity/'
os.makedirs(disparity_dir,exist_ok=True)
destination_dir='D:/Downloads/Analysed/DESTINATION/'
#os.makedirs(destination_dir,exist_ok=True)
#Directory to simply experiment around
#root_dir='D:/Downloads/Analysed/E2 - Copy/'
#classes=next(os.walk(root_dir))[1]
#Extension list is the list of file types available in the dataset.
extension_list=['.jpg', '.xml', '.jpeg', '.png', '', '.JPG', '.json', '.webp', '.html', '.txt', '.cms', '.gif', '.jfif']
#Image extension list contains the types of images available in the dataset
#.gif removed since model doesn't read it
image_extension_list=['.jpg', '.png', '.jpeg', '.JPG', '.webp', '.cms', '.jfif'] 
#Annotation extension list contains the types of annotations available in the dataset
annotation_extension_list=['.xml', '.txt', '.json']
#label_folder_list contains the folder names available in some classes which 
#contain the annotations of that class
label_folder_list=['labes','label','annotation','annotations','labels']
forbidden=['train','test']